<a href="https://colab.research.google.com/github/barzansaeedpour/advanced-python-notebooks/blob/main/SOLID_Principles_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLID Principles with Python
<div dir="rtl">
<h1>
اصول SOLID با پایتون
</h1>
</div>

<div dir="rtl">
اصول SOLID در برنامه نویسی توسط آقای «رابرت سی. مارتین» (Robert C. Martin) در مقاله‌ای به نام «اصول طراحی و الگوهای طراحی» (Design Principles and Design Patterns) در سال ۱۳۸۲ شمسی (۲۰۰۰ میلادی) معرفی شدند. این مفاهیم بعدا توسط «مایکل فدرز» (Michael Feathers) گسترش یافتند. آقای فدرز کسی بود که کلمه مخفف «سالید» (SOLID) را ابدع کرد و برای اولین بار استفاده کرد و در طول ۲۰ سال گذشته، این اصول پنج‌گانه از طریق اصلاح راه و روش نوشتن نرم‌افزارها، دنیای برنامه‌نویسی شی‌گرایانه را دچار تغییر و تحول بزرگی کردند. در این نوت بوک به اصول سالید در برنامه نویسی خواهیم پرداخت و برای هریک با نوشتن نمونه کد مثال‌هایی ارائه می‌دهیم.
</div>